<a href="https://colab.research.google.com/github/hannapalya/syndromic-clustering/blob/main/MIMIC_Whole_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

In [ ]:
# Function to load and extract text from a .docx file
def load_docx(file_path):
    doc = Document(file_path)
    data = []
    for table in doc.tables:
        for row in table.rows:
            # Extract the cells' text
            cells = [cell.text for cell in row.cells]
            data.append(cells)
    return data

In [ ]:
# Function to identify if the complaint is related to tuberculosis
def is_tuberculosis_related(complaint):
    keywords = ['tuber']
    return any(re.search(keyword, complaint.lower()) for keyword in keywords)


In [ ]:
import pandas as pd
# Load the document
data = pd.read_csv('/content/BERT_MIMIC_whole_clustered_dbscan_eps_80_4.csv')  # Replace with your actual file path



In [ ]:
data

In [ ]:
# Assuming the Chief Complaint is in the 4th column and Cluster Label in the 5th column
chief_complaints = data['chiefcomplaint']
cluster_labels = data['Cluster']


In [ ]:
len(cluster_labels)

In [ ]:
print(chief_complaints)

In [ ]:
print(cluster_labels)

# Silhouette score

In [ ]:
#!pip install gower
import pandas as pd
import gower
from sklearn.metrics import silhouette_samples

# Assuming 'data_vectorized' has a 'date' column for timestamps and a 'cluster_labels' column
# Drop columns that should not be part of clustering
data_for_clustering = data

# Convert 'date' column to datetime if not already
#data_for_clustering['date'] = pd.to_datetime(data_for_clustering['date'])

# Group by week using pandas Grouper
#weekly_groups = data_for_clustering.groupby(pd.Grouper(key='date', freq='W-WED'))


In [ ]:
#!pip install gower
import pandas as pd
import gower
from sklearn.metrics import silhouette_samples

# Assuming 'data' is the full dataset containing the 'date', 'cc', and 'cluster' columns
# Drop the 'date', 'cc', and 'cluster' columns for clustering
data_for_clustering = data.drop(['subject_id', 'stay_id',	'chiefcomplaint'	, 'Cluster'], axis=1)
data_for_clustering = data_for_clustering.dropna()
filtered_data = data.loc[data_for_clustering.index]

#data_for_clustering = data.drop(['Cluster', 'cc'], axis=1)
# Compute Gower distance for the entire dataset
gower_distances = gower.gower_matrix(data_for_clustering)

# Get cluster labels for the entire dataset
cluster_labels = data.loc[data_for_clustering.index, 'Cluster'].values
# Compute silhouette scores, ignoring noise points (-1 if you have any noise labels)
mask = cluster_labels != -2

# Check if there are at least 2 unique clusters after applying the mask
unique_labels = set(cluster_labels[mask])
if len(unique_labels) >= 2 and mask.sum() > 0:  # Proceed only if there are at least 2 unique clusters and samples remaining
    silhouette_vals = silhouette_samples(gower_distances[mask][:, mask], cluster_labels[mask], metric="precomputed")

    # Initialize dictionary to store the silhouette scores per cluster
    cluster_silhouette_scores = {}

    # Loop through each unique cluster and calculate the mean silhouette score for that cluster
    for cluster in unique_labels:
        if cluster != -2:  # Skip noise points
            # Get the silhouette scores for points in this cluster
            cluster_silhouette_vals = silhouette_vals[cluster_labels[mask] == cluster]

            # Compute the mean silhouette score for this cluster
            mean_silhouette_score = cluster_silhouette_vals.mean()
            cluster_silhouette_scores[cluster] = mean_silhouette_score
            print(f"Cluster: {cluster}, Silhouette Score: {mean_silhouette_score}")

    # Extract the cc's for clusters with a silhouette score >= 0
    cc_per_cluster = {}
    for cluster, score in cluster_silhouette_scores.items():
        if score >= -1:
            # Use data to select cc values for the current cluster
            cc_values = filtered_data.loc[cluster_labels == cluster, 'chiefcomplaint'].values
            cc_per_cluster[cluster] = cc_values

else:
    # Handle the case where there are less than 2 unique clusters or no samples after applying the mask
    print("Insufficient unique clusters or no samples found after filtering noise points.")
    cc_per_cluster = {}  # Assign an empty dictionary

# Print the clusters with silhouette score >= 0 and their cc values
for cluster, cc_values in cc_per_cluster.items():
    print(f"  Cluster {cluster}: cc values = {cc_values}")